In [1]:
# Importing packages 
!pip install numpy==1.24.3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
import json
import urllib.request
import urllib.parse
from tqdm import tqdm
import re
import string

from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

from num2words import num2words
from newscatcherapi import NewsCatcherApiClient

from collections import defaultdict
from wordcloud import WordCloud

# enabling inline plots in Jupyter
%matplotlib inline

!pip install bertopic
!pip install sentence-transformers
# disabling verbose messages from datasets library
from datasets import logging
logging.set_verbosity_error()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stilling/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/stilling/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/stilling/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/stilling/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


/opt/anaconda3/envs/berttopic_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pip show numpy

Name: numpy
Version: 1.24.3
Summary: Fundamental package for array computing in Python
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD-3-Clause
Location: /opt/anaconda3/envs/berttopic_env/lib/python3.10/site-packages
Requires: 
Required-by: bertopic, contourpy, datasets, hdbscan, matplotlib, numba, pandas, scikit-learn, scipy, seaborn, transformers, umap-learn, wordcloud
Note: you may need to restart the kernel to use updated packages.


### Bring in observational data for inspection and preprocesing 

I have two data sets - one with specified sources from federal government and provincal government and one with general sources defined by countries USA, CANADA or Great Britan

In [92]:



#df_ltd = pd.read_csv('/Users/stilling/Desktop/bert_topic_limited/top_articles.csv')
df_ltd = pd.read_csv('/Users/stilling/Desktop/bert_topic/final_data_analysis.csv')

In [93]:
df_ltd.head()

#drop includes, excludes, reference_set, Unnamed: 0.1, Unnamed: 0, source_weight, rank_normalized, _score_normalized
df_ltd = df_ltd.drop(['includes', 'excludes', 'reference_set', 'Unnamed: 0.1', 'Unnamed: 0', 'source_weight', 'rank_normalized', '_score_normalized'], axis=1)

In [94]:
#drop text_preprossed	articles_pre_stem 
df_ltd = df_ltd.drop(['text_preprossed', 'articles_pre_stem'], axis=1)

In [95]:
#drop observations with a score less than 20
df_ltd = df_ltd[df_ltd['_score'] >= 20]

In [96]:
#shape of the data
df_ltd.shape

(13249, 18)

In [97]:
df_ltd.head()

,title,author,published_date,link,clean_url,summary,rank,country,authors,is_opinion,_score,date_range,summary_no_phrase,lowcase_summary,summary_cleaned,summary_no_whitespace,summary_no_stopwords,final_classification
0,Trudeau announces help for homeless in Vancouv...,The Canadian Press,2019-02-11,https://nationalpost.com/pmn/news-pmn/canada-n...,nationalpost.com,"This advertisement has not loaded yet, but you...",1127,CA,The Canadian Press,False,87.253510,2019-01-01 to 2019-03-31,His government is providing funding to the Van...,his government is providing funding to the van...,his government is providing funding to the van...,his government is providing funding to the van...,government providing funding vancouver afforda...,True
1,Prime Minister announces investments for safer...,markets.businessinsider.com,2019-02-11,https://markets.businessinsider.com/news/stock...,businessinsider.com,"VANCOUVER, Feb. 11, 2019 /CNW/ - Every Canadia...",106,US,markets.businessinsider.com,False,87.161575,2019-01-01 to 2019-03-31,", Feb. 11, 2019 // - Every Canadian deserves t...",", feb. 11, 2019 // - every canadian deserves t...",feb every canadian deserves to have a safe and...,feb every canadian deserves to have a safe and...,feb every canadian deserves safe affordable pl...,True
2,Trudeau announces help for Vancouver homeless ...,The Canadian Press,2019-02-12,https://www.cbc.ca/news/canada/british-columbi...,cbc.ca,"Prime Minister Justin Trudeau, right, walks wi...",300,CA,The Canadian Press,False,83.397830,2019-01-01 to 2019-03-31,"Prime Minister Justin Trudeau, right, walks wi...","prime minister justin trudeau, right, walks wi...",prime minister justin trudeau right walks with...,prime minister justin trudeau right walks with...,prime minister justin trudeau right walks rich...,True
3,Feds to fund 3 Vancouver temporary modular hou...,The Canadian Press,2019-02-12,https://globalnews.ca/news/4951038/vancouver-t...,globalnews.ca,The federal government announced plans Monday ...,854,CA,The Canadian Press,False,83.028340,2019-01-01 to 2019-03-31,The federal government announced plans Monday ...,the federal government announced plans monday ...,the federal government announced plans monday ...,the federal government announced plans monday ...,federal government announced plans monday deve...,True
4,Federal fund for rental units won't fix housin...,Robyn Miller,2019-03-20,https://www.cbc.ca/news/canada/ottawa/rental-c...,cbc.ca,"Coun. Catherine McKenney, in orange, speaks at...",300,CA,Robyn Miller,False,82.780230,2019-01-01 to 2019-03-31,"Coun. Catherine McKenney, in orange, speaks at...","coun. catherine mckenney, in orange, speaks at...",coun catherine mckenney in orange speaks at an...,coun catherine mckenney in orange speaks at an...,coun catherine mckenney orange speaks affordab...,True


----

## Bert topic modelling on the data set


In [11]:
!pip install bertopic

In [12]:

!pip install sentence-transformers



In [98]:
stop_terms = ["housing", "home", "canadian", "housing price", "new" "housing affordability", "crisis", "say", 'said' "affordable", "say", "said", "government", "journalist", "account", "canadian journalist", "account", "canadian housing", "canadian housing price", "canadian housing affordability", "canadian housing crisis", "canadian housing market"]


In [99]:
noise_patterns = [
    r"\bcanadian press\b", r"\bcanada account\b", r"\barticle canada\b", r"\barticle canada account\b",
    r"\bnew york\b", r"\byork time\b", r"\bnew york time\b", r"\bprint edition\b", r"\belectronic replica\b",
    r"\breplica print\b", r"\breplica print edition\b", r"\btime crossword\b", r"\byork time crossword\b",
    r"\bcity canada\b", r"\bepaper electronic\b", r"\bepaper electronic replica\b", r"\byear ago\b",
    r"\bunited state\b", r"\bpeople live\b", r"\bnova scotia\b", r"\bexecutive director\b",
    r"\bpeople experience\b", r"\bpast year\b", r"\breal estate\b", r"\bglobal news\b",
    r"\bnineteen pandemic\b", r"\banalysis awardwinning\b", r"\banalysis awardwinning journalist\b",
    r"\bbehindthescenes analysis\b", r"\bbehindthescenes analysis awardwinning\b", r"\bcanada article\b",
    r"\bexclusive access\b", r"\bshare download\b", r"\bdownload comment\b", r"\bshare download comment\b",
    r"\bawardwinning journalist generation\b", r"\bjournalist generation\b", r"\bjournalist generation journalist\b",
    r"\bcomment behindthescenes\b", r"\bcomment behindthescenes analysis\b", r"\bdownload comment behindthescenes\b",
    r"\bedition journalist\b", r"\bedition journalist generation\b", r"\baccess article\b",
    r"\bexperience access\b", r"\bconversation comment\b", r"\bjoin conversation comment\b",
    r"\bcity canada account\b", r"\blevel government\b", r"\bfavourite author\b", r"\bcanada mortgage\b",
    r"\bminister justin\b", r"\bedition journalist generation\b", r"\barticle canada\b", r"\barticle canada account\b",
    r"\bcity canada\b", r"\bgeneration journalist\b", r"\bjournalist generation\b", r"\bjournalist generation journalist\b",
    r"\bminister justin\b", r"\bcity canada account\b", r"\bedition journalist\b", r"\bedition journalist generation\b",
    r"\barticle canada\b", r"\barticle canada account\b"
]

# Remove any potential duplicates by converting to a set and back to list
noise_patterns = list(set(noise_patterns))


In [122]:
#install tqdm
!pip install tqdm

# Compile the regex patterns into a single regex object for efficiency
noise_regex = re.compile('|'.join(noise_patterns), flags=re.IGNORECASE)

# Function to remove noise terms
def remove_noise(text):
    if not isinstance(text, str):
        return text
    # Replace noise terms with an empty string
    return noise_regex.sub("", text).strip()

# Apply the cleaning function to the 'summary_lemmatized' column
tqdm.pandas(desc="Removing noise from preprossed summaries")
df_ltd['cleaned'] = df_ltd['summary_no_stopwords'].progress_apply(remove_noise)

# Verify the changes
print(df_ltd[['cleaned']].head())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Removing noise from preprossed summaries: 100%|██████████| 13249/13249 [00:04<00:00, 2742.16it/s]

                                             cleaned
0  government providing funding vancouver afforda...
1  feb every canadian deserves safe affordable pl...
2  prime  trudeau right walks richard lee liberal...
3  federal government announced plans monday deve...
4  coun catherine mckenney orange speaks affordab...


In [123]:


# Define your custom stop words to remov
# Function to remove stop words
def remove_stopwords(text):
    if not isinstance(text, str):  # Handle non-string cases
        return ""
    # Split text into words and filter out stop words
    filtered_words = [word for word in text.split() if word.lower() not in stop_terms]
    return ' '.join(filtered_words)  # Rejoin the words into a single string

# Apply the function to the column
df_ltd['cleaned'] = df_ltd['cleaned'].apply(remove_stopwords)

# Verify the results
print(df_ltd['cleaned'].head())


0    providing funding vancouver affordable agency ...
1    feb every deserves safe affordable place call ...
2    prime trudeau right walks richard lee liberal ...
3    federal announced plans monday develop three p...
4    coun catherine mckenney orange speaks affordab...
Name: cleaned, dtype: object


In [124]:
# Extract the cleaned text column
documents = df_ltd['cleaned'].tolist()


In [125]:
pip install numpy==1.24.3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [126]:
!pip show numpy


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: numpy
Version: 1.24.3
Summary: Fundamental package for array computing in Python
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD-3-Clause
Location: /opt/anaconda3/envs/berttopic_env/lib/python3.10/site-packages
Requires: 
Required-by: bertopic, contourpy, datasets, hdbscan, matplotlib, numba, pandas, scikit-learn, scipy, seaborn, transformers, umap-learn, wordcloud


In [127]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer


# Initialize BERTopic
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Compact and efficient embedding model
topic_model = BERTopic(embedding_model=embedding_model)

# Fit the BERTopic model
topics, probs = topic_model.fit_transform(documents)

# Add topics to the dataframe
df_ltd['topic'] = topics


In [153]:
# Get topic information
topic_data = topic_model.get_topic_info()
print(topic_data)

#save topic data to csv
topic_data.to_csv('/Users/stilling/Desktop/bert_topic/primary_topic_data.csv')


     Topic  Count                                           Name  \
0       -1   4751                 -1_affordable_new_rental_units   
1        0    444          0_homelessness_homeless_people_health   
2        1    316  1_immigration_immigrants_population_newcomers   
3        2    232  2_students_international_student_universities   
4        3    202               3_trudeau_prime_minister_liberal   
..     ...    ...                                            ...   
204    203     11       203_newmarket_taylor_town_newmarkettoday   
205    204     11             204_fiscal_deficits_fall_statement   
206    205     11             205_casey_richter_tkachuk_homeless   
207    206     10              206_artistic_ft_dogwood_rendering   
208    207     10              207_rafuse_dhanji_chender_kabalen   

                                        Representation  \
0    [affordable, new, rental, units, one, people, ...   
1    [homelessness, homeless, people, health, exper...   
2    [imm

In [129]:
# Visualize topics
topic_model.visualize_topics()


In [130]:
#show topics display topics
df_ltd.head()

,title,author,published_date,link,clean_url,summary,rank,country,authors,is_opinion,_score,date_range,summary_no_phrase,lowcase_summary,summary_cleaned,summary_no_whitespace,summary_no_stopwords,final_classification,cleaned,topic
0,Trudeau announces help for homeless in Vancouv...,The Canadian Press,2019-02-11,https://nationalpost.com/pmn/news-pmn/canada-n...,nationalpost.com,"This advertisement has not loaded yet, but you...",1127,CA,The Canadian Press,False,87.253510,2019-01-01 to 2019-03-31,His government is providing funding to the Van...,his government is providing funding to the van...,his government is providing funding to the van...,his government is providing funding to the van...,government providing funding vancouver afforda...,True,providing funding vancouver affordable agency ...,67
1,Prime Minister announces investments for safer...,markets.businessinsider.com,2019-02-11,https://markets.businessinsider.com/news/stock...,businessinsider.com,"VANCOUVER, Feb. 11, 2019 /CNW/ - Every Canadia...",106,US,markets.businessinsider.com,False,87.161575,2019-01-01 to 2019-03-31,", Feb. 11, 2019 // - Every Canadian deserves t...",", feb. 11, 2019 // - every canadian deserves t...",feb every canadian deserves to have a safe and...,feb every canadian deserves to have a safe and...,feb every canadian deserves safe affordable pl...,True,feb every deserves safe affordable place call ...,-1
2,Trudeau announces help for Vancouver homeless ...,The Canadian Press,2019-02-12,https://www.cbc.ca/news/canada/british-columbi...,cbc.ca,"Prime Minister Justin Trudeau, right, walks wi...",300,CA,The Canadian Press,False,83.397830,2019-01-01 to 2019-03-31,"Prime Minister Justin Trudeau, right, walks wi...","prime minister justin trudeau, right, walks wi...",prime minister justin trudeau right walks with...,prime minister justin trudeau right walks with...,prime minister justin trudeau right walks rich...,True,prime trudeau right walks richard lee liberal ...,14
3,Feds to fund 3 Vancouver temporary modular hou...,The Canadian Press,2019-02-12,https://globalnews.ca/news/4951038/vancouver-t...,globalnews.ca,The federal government announced plans Monday ...,854,CA,The Canadian Press,False,83.028340,2019-01-01 to 2019-03-31,The federal government announced plans Monday ...,the federal government announced plans monday ...,the federal government announced plans monday ...,the federal government announced plans monday ...,federal government announced plans monday deve...,True,federal announced plans monday develop three p...,67
4,Federal fund for rental units won't fix housin...,Robyn Miller,2019-03-20,https://www.cbc.ca/news/canada/ottawa/rental-c...,cbc.ca,"Coun. Catherine McKenney, in orange, speaks at...",300,CA,Robyn Miller,False,82.780230,2019-01-01 to 2019-03-31,"Coun. Catherine McKenney, in orange, speaks at...","coun. catherine mckenney, in orange, speaks at...",coun catherine mckenney in orange speaks at an...,coun catherine mckenney in orange speaks at an...,coun catherine mckenney orange speaks affordab...,True,coun catherine mckenney orange speaks affordab...,28


In [131]:
#save the data
df_ltd.to_csv('/Users/stilling/Desktop/bert_topic/topic_modelled.csv')

In [132]:
df_ltd.head()

,title,author,published_date,link,clean_url,summary,rank,country,authors,is_opinion,_score,date_range,summary_no_phrase,lowcase_summary,summary_cleaned,summary_no_whitespace,summary_no_stopwords,final_classification,cleaned,topic
0,Trudeau announces help for homeless in Vancouv...,The Canadian Press,2019-02-11,https://nationalpost.com/pmn/news-pmn/canada-n...,nationalpost.com,"This advertisement has not loaded yet, but you...",1127,CA,The Canadian Press,False,87.253510,2019-01-01 to 2019-03-31,His government is providing funding to the Van...,his government is providing funding to the van...,his government is providing funding to the van...,his government is providing funding to the van...,government providing funding vancouver afforda...,True,providing funding vancouver affordable agency ...,67
1,Prime Minister announces investments for safer...,markets.businessinsider.com,2019-02-11,https://markets.businessinsider.com/news/stock...,businessinsider.com,"VANCOUVER, Feb. 11, 2019 /CNW/ - Every Canadia...",106,US,markets.businessinsider.com,False,87.161575,2019-01-01 to 2019-03-31,", Feb. 11, 2019 // - Every Canadian deserves t...",", feb. 11, 2019 // - every canadian deserves t...",feb every canadian deserves to have a safe and...,feb every canadian deserves to have a safe and...,feb every canadian deserves safe affordable pl...,True,feb every deserves safe affordable place call ...,-1
2,Trudeau announces help for Vancouver homeless ...,The Canadian Press,2019-02-12,https://www.cbc.ca/news/canada/british-columbi...,cbc.ca,"Prime Minister Justin Trudeau, right, walks wi...",300,CA,The Canadian Press,False,83.397830,2019-01-01 to 2019-03-31,"Prime Minister Justin Trudeau, right, walks wi...","prime minister justin trudeau, right, walks wi...",prime minister justin trudeau right walks with...,prime minister justin trudeau right walks with...,prime minister justin trudeau right walks rich...,True,prime trudeau right walks richard lee liberal ...,14
3,Feds to fund 3 Vancouver temporary modular hou...,The Canadian Press,2019-02-12,https://globalnews.ca/news/4951038/vancouver-t...,globalnews.ca,The federal government announced plans Monday ...,854,CA,The Canadian Press,False,83.028340,2019-01-01 to 2019-03-31,The federal government announced plans Monday ...,the federal government announced plans monday ...,the federal government announced plans monday ...,the federal government announced plans monday ...,federal government announced plans monday deve...,True,federal announced plans monday develop three p...,67
4,Federal fund for rental units won't fix housin...,Robyn Miller,2019-03-20,https://www.cbc.ca/news/canada/ottawa/rental-c...,cbc.ca,"Coun. Catherine McKenney, in orange, speaks at...",300,CA,Robyn Miller,False,82.780230,2019-01-01 to 2019-03-31,"Coun. Catherine McKenney, in orange, speaks at...","coun. catherine mckenney, in orange, speaks at...",coun catherine mckenney in orange speaks at an...,coun catherine mckenney in orange speaks at an...,coun catherine mckenney orange speaks affordab...,True,coun catherine mckenney orange speaks affordab...,28


In [133]:
#visualise the topic 1 over time
df_ltd['published_date'] = pd.to_datetime(df_ltd['published_date'])

In [134]:
# Get topic information
topic_info = topic_model.get_topic_info()

# Filter topics 0-20
filtered_topics = topic_info[topic_info['Topic'].between(0, 50)]
print(filtered_topics)


    Topic  Count                                           Name  \
1       0    444          0_homelessness_homeless_people_health   
2       1    316  1_immigration_immigrants_population_newcomers   
3       2    232  2_students_international_student_universities   
4       3    202               3_trudeau_prime_minister_liberal   
5       4    186                   4_budget_freeland_fiscal_tax   
6       5    168        5_mortgage_quarter_affordability_prices   
7       6    165              6_seniors_homes_community_society   
8       7    145                  7_rental_rent_rentals_average   
9       8    138      8_calgary_calgarians_calgarys_journalists   
10      9    132              9_council_tiny_county_development   
11     10    126        10_median_markets_unaffordable_multiple   
12     11    123          11_accelerator_initiatives_homes_fund   
13     12    123          12_supply_affordability_prices_demand   
14     13    120           13_party_riding_candidate_candidate

In [135]:
# Create a new DataFrame for unclassified articles
unclassified_df = df_ltd[df_ltd['topic'] == -1].copy()

# Ensure 'cleaned' column exists in the new DataFrame
unclassified_docs = unclassified_df['cleaned'].tolist()


-----
# handling unassigned


In [136]:
# Initialize a new BERTopic instance
topic_model_unclassified = BERTopic(embedding_model=embedding_model, min_topic_size=5, nr_topics="auto")

# Fit BERTopic on unclassified documents
new_topics, new_probs = topic_model_unclassified.fit_transform(unclassified_docs)

# Add new topics back to the unclassified DataFrame
unclassified_df['new_topic'] = new_topics


In [152]:
# Get topic information
unclass_topic_data = topic_model_unclassified.get_topic_info()
print(unclass_topic_data)

#save topic data to csv
unclass_topic_data.to_csv('/Users/stilling/Desktop/bert_topic/unclass_topic_info.csv')


     Topic  Count                                      Name  \
0       -1   2084            -1_affordable_people_new_units   
1        0   1019     0_affordable_support_new_homelessness   
2        1    233                    1_rent_cent_per_rental   
3        2     34          2_quebec_dhabitation_montréal_de   
4        3     27  3_brunswick_fredericton_minister_moncton   
..     ...    ...                                       ...   
145    144      5              144_zero_three_carleton_nine   
146    145      5              145_lee_metro_heat_vancouver   
147    146      5       146_parksville_qualicum_lake_barges   
148    147      5          147_agree_swirling_pierre_update   
149    148      5            148_crowding_haider_census_tol   

                                        Representation  \
0    [affordable, people, new, units, canada, renta...   
1    [affordable, support, new, homelessness, city,...   
2    [rent, cent, per, rental, tenants, market, lan...   
3    [quebe

In [138]:
unclassified_df.head()

,title,author,published_date,link,clean_url,summary,rank,country,authors,is_opinion,...,date_range,summary_no_phrase,lowcase_summary,summary_cleaned,summary_no_whitespace,summary_no_stopwords,final_classification,cleaned,topic,new_topic
1,Prime Minister announces investments for safer...,markets.businessinsider.com,2019-02-11,https://markets.businessinsider.com/news/stock...,businessinsider.com,"VANCOUVER, Feb. 11, 2019 /CNW/ - Every Canadia...",106,US,markets.businessinsider.com,False,...,2019-01-01 to 2019-03-31,", Feb. 11, 2019 // - Every Canadian deserves t...",", feb. 11, 2019 // - every canadian deserves t...",feb every canadian deserves to have a safe and...,feb every canadian deserves to have a safe and...,feb every canadian deserves safe affordable pl...,True,feb every deserves safe affordable place call ...,-1,0
5,Fact check: Will the NDP's housing plan ‘bankr...,Terry Pedwell,2019-01-24,https://globalnews.ca/news/4882936/fact-check-...,globalnews.ca,'Their front-end loading plan is a recipe to b...,854,CA,"Terry Pedwell,The Canadian Press",False,...,2019-01-01 to 2019-03-31,'Their front-end loading plan is a recipe to b...,'their front-end loading plan is a recipe to b...,their frontend loading plan is a recipe to ban...,their frontend loading plan is a recipe to ban...,frontend loading plan recipe bankrupt housing ...,True,frontend loading plan recipe bankrupt provider...,-1,0
6,Governments of Canada and British Columbia Lau...,Department Of Finance Canada,2019-03-15,https://www.canada.ca/en/department-finance/ne...,canada.ca,Governments of Canada and British Columbia Lau...,544,CA,"Department Of Finance Canada,Government of Canada",False,...,2019-01-01 to 2019-03-31,Governments of Canada and British Columbia Lau...,governments of canada and british columbia lau...,governments of canada and british columbia lau...,governments of canada and british columbia lau...,governments canada british columbia launch exp...,True,governments canada british columbia launch exp...,-1,-1
8,Baloney Meter: Is NDP housing plan 'a recipe t...,Terry Pedwell,2019-01-24,https://infotel.ca/newsitem/baloney-meter-hous...,infotel.ca,Baloney Meter: Is NDP housing plan 'a recipe t...,47478,CA,Terry Pedwell,False,...,2019-01-01 to 2019-03-31,Baloney Meter: Is housing plan 'a recipe to b...,baloney meter: is housing plan 'a recipe to b...,baloney meter is housing plan a recipe to bank...,baloney meter is housing plan a recipe to bank...,baloney meter housing plan recipe bankrupt hou...,True,baloney meter plan recipe bankrupt providers f...,-1,0
10,Fact check: Is NDP housing plan 'a recipe to b...,Terry Pedwell,2019-01-24,https://www.ctvnews.ca/politics/fact-check-is-...,ctvnews.ca,"OTTAWA -- ""Their front-end loading plan is a r...",909,CA,Terry Pedwell,False,...,2019-01-01 to 2019-03-31,"-- ""Their front-end loading plan is a recipe ...","-- ""their front-end loading plan is a recipe ...",their frontend loading plan is a recipe to ban...,their frontend loading plan is a recipe to ban...,frontend loading plan recipe bankrupt housing ...,True,frontend loading plan recipe bankrupt provider...,-1,0


In [139]:
#drop the topic column from the unclassified data
unclassified_df = unclassified_df.drop(['topic'], axis=1)

In [140]:
#rename the new_topic column to topic
#unclassified_df = unclassified_df.rename(columns={'new_topic': 'topic'})

In [144]:

# Add a new column called 'new_topic' with NaN values
df_ltd['new_topic'] = np.nan


In [145]:
df_ltd.head()

,title,author,published_date,link,clean_url,summary,rank,country,authors,is_opinion,...,date_range,summary_no_phrase,lowcase_summary,summary_cleaned,summary_no_whitespace,summary_no_stopwords,final_classification,cleaned,topic,new_topic
0,Trudeau announces help for homeless in Vancouv...,The Canadian Press,2019-02-11,https://nationalpost.com/pmn/news-pmn/canada-n...,nationalpost.com,"This advertisement has not loaded yet, but you...",1127,CA,The Canadian Press,False,...,2019-01-01 to 2019-03-31,His government is providing funding to the Van...,his government is providing funding to the van...,his government is providing funding to the van...,his government is providing funding to the van...,government providing funding vancouver afforda...,True,providing funding vancouver affordable agency ...,67,NaN
1,Prime Minister announces investments for safer...,markets.businessinsider.com,2019-02-11,https://markets.businessinsider.com/news/stock...,businessinsider.com,"VANCOUVER, Feb. 11, 2019 /CNW/ - Every Canadia...",106,US,markets.businessinsider.com,False,...,2019-01-01 to 2019-03-31,", Feb. 11, 2019 // - Every Canadian deserves t...",", feb. 11, 2019 // - every canadian deserves t...",feb every canadian deserves to have a safe and...,feb every canadian deserves to have a safe and...,feb every canadian deserves safe affordable pl...,True,feb every deserves safe affordable place call ...,-1,NaN
2,Trudeau announces help for Vancouver homeless ...,The Canadian Press,2019-02-12,https://www.cbc.ca/news/canada/british-columbi...,cbc.ca,"Prime Minister Justin Trudeau, right, walks wi...",300,CA,The Canadian Press,False,...,2019-01-01 to 2019-03-31,"Prime Minister Justin Trudeau, right, walks wi...","prime minister justin trudeau, right, walks wi...",prime minister justin trudeau right walks with...,prime minister justin trudeau right walks with...,prime minister justin trudeau right walks rich...,True,prime trudeau right walks richard lee liberal ...,14,NaN
3,Feds to fund 3 Vancouver temporary modular hou...,The Canadian Press,2019-02-12,https://globalnews.ca/news/4951038/vancouver-t...,globalnews.ca,The federal government announced plans Monday ...,854,CA,The Canadian Press,False,...,2019-01-01 to 2019-03-31,The federal government announced plans Monday ...,the federal government announced plans monday ...,the federal government announced plans monday ...,the federal government announced plans monday ...,federal government announced plans monday deve...,True,federal announced plans monday develop three p...,67,NaN
4,Federal fund for rental units won't fix housin...,Robyn Miller,2019-03-20,https://www.cbc.ca/news/canada/ottawa/rental-c...,cbc.ca,"Coun. Catherine McKenney, in orange, speaks at...",300,CA,Robyn Miller,False,...,2019-01-01 to 2019-03-31,"Coun. Catherine McKenney, in orange, speaks at...","coun. catherine mckenney, in orange, speaks at...",coun catherine mckenney in orange speaks at an...,coun catherine mckenney in orange speaks at an...,coun catherine mckenney orange speaks affordab...,True,coun catherine mckenney orange speaks affordab...,28,NaN


In [146]:
#drop the unclassified data from the main data
#make a copy of the main data
df_topic = df_ltd.copy()


In [147]:
df_topic = df_topic[df_topic['topic'] != -1]

In [149]:
#merge the unclassified data with the main data
df_topic = pd.concat([df_topic, unclassified_df])

#reset the index
df_topic = df_topic.reset_index(drop=True)



In [150]:
df_topic.head()

,title,author,published_date,link,clean_url,summary,rank,country,authors,is_opinion,...,date_range,summary_no_phrase,lowcase_summary,summary_cleaned,summary_no_whitespace,summary_no_stopwords,final_classification,cleaned,topic,new_topic
0,Trudeau announces help for homeless in Vancouv...,The Canadian Press,2019-02-11,https://nationalpost.com/pmn/news-pmn/canada-n...,nationalpost.com,"This advertisement has not loaded yet, but you...",1127,CA,The Canadian Press,False,...,2019-01-01 to 2019-03-31,His government is providing funding to the Van...,his government is providing funding to the van...,his government is providing funding to the van...,his government is providing funding to the van...,government providing funding vancouver afforda...,True,providing funding vancouver affordable agency ...,67,NaN
1,Trudeau announces help for Vancouver homeless ...,The Canadian Press,2019-02-12,https://www.cbc.ca/news/canada/british-columbi...,cbc.ca,"Prime Minister Justin Trudeau, right, walks wi...",300,CA,The Canadian Press,False,...,2019-01-01 to 2019-03-31,"Prime Minister Justin Trudeau, right, walks wi...","prime minister justin trudeau, right, walks wi...",prime minister justin trudeau right walks with...,prime minister justin trudeau right walks with...,prime minister justin trudeau right walks rich...,True,prime trudeau right walks richard lee liberal ...,14,NaN
2,Feds to fund 3 Vancouver temporary modular hou...,The Canadian Press,2019-02-12,https://globalnews.ca/news/4951038/vancouver-t...,globalnews.ca,The federal government announced plans Monday ...,854,CA,The Canadian Press,False,...,2019-01-01 to 2019-03-31,The federal government announced plans Monday ...,the federal government announced plans monday ...,the federal government announced plans monday ...,the federal government announced plans monday ...,federal government announced plans monday deve...,True,federal announced plans monday develop three p...,67,NaN
3,Federal fund for rental units won't fix housin...,Robyn Miller,2019-03-20,https://www.cbc.ca/news/canada/ottawa/rental-c...,cbc.ca,"Coun. Catherine McKenney, in orange, speaks at...",300,CA,Robyn Miller,False,...,2019-01-01 to 2019-03-31,"Coun. Catherine McKenney, in orange, speaks at...","coun. catherine mckenney, in orange, speaks at...",coun catherine mckenney in orange speaks at an...,coun catherine mckenney in orange speaks at an...,coun catherine mckenney orange speaks affordab...,True,coun catherine mckenney orange speaks affordab...,28,NaN
4,Governments of Canada and Ontario celebrate ne...,markets.businessinsider.com,2019-03-12,https://markets.businessinsider.com/news/stock...,businessinsider.com,"THUNDER BAY, ON, March 12, 2019 /CNW/ - More p...",106,US,markets.businessinsider.com,False,...,2019-01-01 to 2019-03-31,", , March 12, 2019 // - More people will now ...",", , march 12, 2019 // - more people will now ...",more people will now have a place to call home...,more people will now have a place to call home...,people place call home northwestern ontario fe...,True,people place call northwestern ontario federal...,42,NaN


In [154]:
#save the data
df_topic.to_csv('/Users/stilling/Desktop/bert_topic/topic_model_full-set.csv')